In [ ]:
# ULTIMATE HUMANOID WALKING TRAINING - ALL LATEST IMPROVEMENTS
# Includes: Basic training, Quick improvement, Curriculum learning, Advanced rewards
# No demo data needed - pure RL from scratch with best methods!

import numpy as np
import gymnasium as gym
import mujoco
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import EvalCallback, CheckpointCallback
import time
import os

print("🚀 ULTIMATE HUMANOID WALKING TRAINING SUITE")
print("=" * 50)
print("✅ All latest improvements loaded!")
print("🎯 Includes: Basic → Quick Improve → Curriculum → Advanced")
print("🚫 No demo data needed - pure RL from scratch!")
print("📊 Tensorboard monitoring included!")


/Users/arhaan17/Coding/Movement_Tracking_Mujoco/venv/lib/python3.13/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/Users/arhaan17/Coding/Movement_Tracking_Mujoco/venv/lib/python3.13/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/Users/arhaan17/Coding/Movement_Tracking_Mujoco/venv/lib/python3.13/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1

✅ New working approach loaded successfully!
🎯 Using step-by-step training: Standing → Walking
🚫 No demo data needed - pure RL from scratch!


In [ ]:
# DEFINE ALL ENVIRONMENT CLASSES (Basic → Improved)
# All environments included directly in notebook

class StandingHumanoidEnv(gym.Env):
    """Phase 1: Learn to stand upright and maintain balance."""
    
    def __init__(self, xml_path="humanoid.xml"):
        super().__init__()
        self.model = mujoco.MjModel.from_xml_path(xml_path)
        self.data = mujoco.MjData(self.model)
        self.viewer = None
        
        obs_dim = self.model.nq + self.model.nv
        self.observation_space = gym.spaces.Box(-np.inf, np.inf, shape=(obs_dim,), dtype=np.float32)
        self.action_space = gym.spaces.Box(-0.3, 0.3, shape=(self.model.nu,), dtype=np.float32)
        
        self.step_count = 0
        
    def reset(self, seed=None):
        super().reset(seed=seed)
        self.data.qpos[:] = 0
        self.data.qpos[2] = 1.3  # Standing height
        self.data.qpos[3] = 1.0  # Upright quaternion
        
        # Add tiny random perturbations
        self.data.qpos[:] += np.random.normal(0, 0.02, size=self.model.nq)
        self.data.qpos[2] = max(1.2, self.data.qpos[2])
        self.data.qvel[:] = 0
        
        mujoco.mj_forward(self.model, self.data)
        self.step_count = 0
        return self._get_obs(), {}
    
    def step(self, action):
        self.data.ctrl[:] = np.clip(action, -0.3, 0.3)
        mujoco.mj_step(self.model, self.data)
        
        obs = self._get_obs()
        reward = self._standing_reward()
        
        terminated = self.data.qpos[2] < 0.5
        truncated = self.step_count >= 500
        self.step_count += 1
        
        return obs, reward, terminated, truncated, {}
    
    def _get_obs(self):
        return np.concatenate([self.data.qpos, self.data.qvel])
    
    def _standing_reward(self):
        height = self.data.qpos[2]
        if height > 1.2:
            height_reward = 2.0
        elif height > 1.0:
            height_reward = 1.0
        elif height > 0.8:
            height_reward = 0.5
        else:
            height_reward = -1.0
        
        stability_bonus = 1.0 if height > 1.0 else 0.0
        control_penalty = -0.001 * np.sum(np.square(self.data.ctrl))
        survival_bonus = 0.1
        
        return height_reward + stability_bonus + control_penalty + survival_bonus
    
    def render(self, mode="human"):
        if self.viewer is None:
            try:
                import mujoco.viewer as viewer
                self.viewer = viewer.launch_passive(self.model, self.data)
            except:
                self.viewer = "disabled"
        if self.viewer != "disabled" and hasattr(self.viewer, 'sync'):
            self.viewer.sync()

class WalkingHumanoidEnv(StandingHumanoidEnv):
    """Phase 2: Learn to walk forward while maintaining balance."""
    
    def __init__(self, xml_path="humanoid.xml"):
        super().__init__(xml_path)
        self.action_space = gym.spaces.Box(-0.5, 0.5, shape=(self.model.nu,), dtype=np.float32)
        
    def step(self, action):
        self.data.ctrl[:] = np.clip(action, -0.5, 0.5)
        mujoco.mj_step(self.model, self.data)
        
        obs = self._get_obs()
        reward = self._walking_reward()
        
        terminated = self.data.qpos[2] < 0.4 or abs(self.data.qpos[1]) > 2.0
        truncated = self.step_count >= 1000
        self.step_count += 1
        
        return obs, reward, terminated, truncated, {}
    
    def _walking_reward(self):
        pos = self.data.qpos
        vel = self.data.qvel
        
        height = pos[2]
        forward_vel = vel[0]
        side_pos = abs(pos[1])
        
        # Height reward (most important)
        if height > 1.1:
            height_reward = 3.0
        elif height > 0.9:
            height_reward = 2.0
        elif height > 0.6:
            height_reward = 1.0
        else:
            height_reward = -2.0
        
        # Forward movement reward
        if forward_vel > 0.2:
            forward_reward = 2.0 * min(forward_vel, 1.5)
        else:
            forward_reward = 0.0
        
        # Penalties and bonuses
        side_penalty = -1.0 * side_pos
        control_penalty = -0.01 * np.sum(np.square(self.data.ctrl))
        survival_bonus = 0.2
        
        return height_reward + forward_reward + side_penalty + control_penalty + survival_bonus

print("✅ Basic environments defined!")

# Test basic standing environment
standing_env = StandingHumanoidEnv()
obs, info = standing_env.reset()
print(f"📊 Standing env - Obs: {obs.shape}, Actions: {standing_env.action_space}")

# Quick standing test
total_reward = 0
for i in range(100):
    action = np.zeros(standing_env.action_space.shape[0])
    obs, reward, terminated, truncated, _ = standing_env.step(action)
    total_reward += reward
    if terminated:
        break

print(f"📈 Standing test: {total_reward/100:.2f} avg reward")
print("✅ Basic environments ready!")


🧪 Testing Standing Environment...
✅ Standing environment created!
📊 Observation shape: (55,)
🎮 Action space: Box(-0.3, 0.3, (21,), float32)
🎯 Goal: Learn to stand upright (height > 1.0m)
📈 Zero control test: 3.10 avg reward
✅ Environment ready for training!


In [ ]:
# ADVANCED IMPROVED WALKING ENVIRONMENT
# Enhanced reward function with curriculum learning support

class ImprovedWalkingEnv(gym.Env):
    """Enhanced walking environment with better reward shaping and curriculum learning"""
    
    def __init__(self, xml_path="humanoid.xml", difficulty=1.0):
        super().__init__()
        self.model = mujoco.MjModel.from_xml_path(xml_path)
        self.data = mujoco.MjData(self.model)
        self.viewer = None
        
        # Difficulty scaling (0.5 = easier, 2.0 = harder)
        self.difficulty = difficulty
        
        obs_dim = self.model.nq + self.model.nv
        self.observation_space = gym.spaces.Box(-np.inf, np.inf, shape=(obs_dim,), dtype=np.float32)
        self.action_space = gym.spaces.Box(-0.8, 0.8, shape=(self.model.nu,), dtype=np.float32)
        
        # Episode parameters
        self.max_episode_steps = int(1500 / difficulty)  # Easier = longer episodes
        self.step_count = 0
        
        # Reward tracking
        self.prev_x_pos = 0.0
        self.cumulative_distance = 0.0
        self.stability_bonus = 0.0
        
    def reset(self, seed=None):
        super().reset(seed=seed)
        
        # Reset to standing with slight randomization
        self.data.qpos[:] = 0
        self.data.qpos[2] = 1.3  # Standing height
        self.data.qpos[3] = 1.0  # Upright quaternion
        
        # Add small perturbations for robustness
        noise_scale = 0.03 / self.difficulty  # Easier = less noise
        self.data.qpos[:] += np.random.normal(0, noise_scale, size=self.model.nq)
        self.data.qpos[2] = max(1.1, self.data.qpos[2])
        
        # Zero velocities with small random component
        self.data.qvel[:] = 0
        self.data.qvel[:] += np.random.normal(0, 0.02, size=self.model.nv)
        
        mujoco.mj_forward(self.model, self.data)
        
        # Reset tracking
        self.step_count = 0
        self.prev_x_pos = self.data.qpos[0]
        self.cumulative_distance = 0.0
        self.stability_bonus = 0.0
        
        return self._get_obs(), {}
    
    def step(self, action):
        # Apply action with scaling
        action_scale = 0.7 / self.difficulty  # Easier = smaller actions
        self.data.ctrl[:] = np.clip(action * action_scale, -0.8, 0.8)
        
        # Step simulation
        mujoco.mj_step(self.model, self.data)
        
        obs = self._get_obs()
        reward = self._compute_enhanced_reward()
        
        # Enhanced termination conditions
        terminated = self._is_terminated()
        truncated = self.step_count >= self.max_episode_steps
        
        self.step_count += 1
        
        # Update tracking
        current_x = self.data.qpos[0]
        forward_progress = max(0, current_x - self.prev_x_pos)
        self.cumulative_distance += forward_progress
        self.prev_x_pos = current_x
        
        return obs, reward, terminated, truncated, {}
    
    def _get_obs(self):
        return np.concatenate([self.data.qpos, self.data.qvel])
    
    def _compute_enhanced_reward(self):
        """Enhanced reward function with better walking incentives"""
        pos = self.data.qpos
        vel = self.data.qvel
        
        # Position and velocity components
        x, y, z = pos[0], pos[1], pos[2]
        vx, vy, vz = vel[0], vel[1], vel[2]
        
        # Quaternion for orientation
        quat = pos[3:7]
        w, qx, qy, qz = quat[0], quat[1], quat[2], quat[3]
        
        reward = 0.0
        
        # 1. FORWARD VELOCITY (primary objective)
        target_speed = 1.2 * self.difficulty
        if vx > 0.3:
            forward_reward = 4.0 * min(vx / target_speed, 1.5)
            # Bonus for maintaining good speed
            if 0.8 <= vx <= 2.0:
                forward_reward += 2.0
        else:
            forward_reward = -0.5  # Penalty for not moving forward
        reward += forward_reward
        
        # 2. HEIGHT STABILITY (crucial for walking)
        target_height = 1.3
        height_diff = abs(z - target_height)
        if height_diff < 0.1:
            height_reward = 3.0
        elif height_diff < 0.3:
            height_reward = 2.0 * (1.0 - height_diff / 0.3)
        else:
            height_reward = -2.0
        reward += height_reward
        
        # 3. UPRIGHT ORIENTATION
        upright_factor = 2 * (w**2 + qz**2) - 1
        upright_reward = 2.0 * max(0, upright_factor)
        reward += upright_reward
        
        # 4. LATERAL STABILITY
        lateral_penalty = -2.0 * (abs(vy) + abs(y))
        reward += lateral_penalty
        
        # 5. GAIT SMOOTHNESS (encourage natural walking)
        action_smoothness = -0.01 * np.sum(np.square(self.data.ctrl))
        reward += action_smoothness
        
        # 6. PROGRESS BONUS
        if self.cumulative_distance > 0:
            progress_bonus = min(self.cumulative_distance * 0.5, 3.0)
            reward += progress_bonus
        
        # 7. STABILITY BONUS (for staying upright over time)
        if z > 1.0 and abs(y) < 0.3:
            self.stability_bonus += 0.01
            reward += min(self.stability_bonus, 1.0)
        
        # 8. STEP SURVIVAL BONUS
        reward += 0.05
        
        # 9. PENALTY SYSTEM
        if z < 0.5:  # Fallen
            reward -= 10.0
        if abs(y) > 2.0:  # Too far sideways
            reward -= 5.0
        if vx < -0.5:  # Moving backward
            reward -= 2.0
        
        # 10. CURRICULUM SCALING
        reward *= (0.5 + 0.5 * self.difficulty)  # Scale rewards with difficulty
        
        return reward
    
    def _is_terminated(self):
        pos = self.data.qpos
        
        # More forgiving termination for easier training
        min_height = 0.4 / self.difficulty
        max_lateral = 3.0 / self.difficulty
        
        if pos[2] < min_height:
            return True
        if abs(pos[1]) > max_lateral:
            return True
        if pos[0] < -3.0:  # Moved too far backward
            return True
            
        return False
    
    def render(self, mode="human"):
        if self.viewer is None:
            try:
                import mujoco.viewer as viewer
                self.viewer = viewer.launch_passive(self.model, self.data)
            except:
                self.viewer = "disabled"
        
        if self.viewer != "disabled" and hasattr(self.viewer, 'sync'):
            self.viewer.sync()

print("✅ Advanced ImprovedWalkingEnv defined!")

# Create directories for models and logs
os.makedirs("models", exist_ok=True)
os.makedirs("logs", exist_ok=True)

print("📁 Model and log directories created")
print("🎯 Ready for training with multiple approaches!")


🚀 PHASE 1: Training Standing Model
Using cpu device
✅ Standing model created
🎯 Training to stand upright...
📊 Small actions (-0.3 to 0.3) for gentle learning


In [ ]:
# TRAINING METHOD 1: BASIC STEP-BY-STEP TRAINING
# Start here if you're new or want the simple approach

def basic_training():
    """Basic step-by-step training: Standing → Walking"""
    print("🚀 BASIC STEP-BY-STEP TRAINING")
    print("=" * 40)
    
    # Phase 1: Standing
    print("🎯 Phase 1: Training Standing Model...")
    standing_env_vec = DummyVecEnv([lambda: Monitor(StandingHumanoidEnv())])
    
    standing_model = PPO(
        "MlpPolicy", standing_env_vec,
        learning_rate=3e-4, n_steps=1024, batch_size=32, n_epochs=5,
        gamma=0.99, verbose=1, tensorboard_log="./logs/basic/",
        policy_kwargs=dict(net_arch=dict(pi=[128, 128], vf=[128, 128]))
    )
    
    standing_model.learn(total_timesteps=50000, progress_bar=True, tb_log_name="basic_standing")
    standing_model.save("basic_standing_model")
    print("✅ Standing model trained!")
    
    # Phase 2: Walking
    print("\n🎯 Phase 2: Training Walking Model...")
    walking_env_vec = DummyVecEnv([lambda: Monitor(WalkingHumanoidEnv())])
    
    try:
        walking_model = PPO.load("basic_standing_model", env=walking_env_vec)
        print("✅ Loaded standing model as base")
    except:
        walking_model = PPO(
            "MlpPolicy", walking_env_vec,
            learning_rate=2e-4, n_steps=2048, batch_size=64, n_epochs=8,
            gamma=0.99, verbose=1, tensorboard_log="./logs/basic/",
            policy_kwargs=dict(net_arch=dict(pi=[256, 256], vf=[256, 256]))
        )
        print("✅ Created new walking model")
    
    walking_model.learn(total_timesteps=100000, progress_bar=True, 
                       tb_log_name="basic_walking", reset_num_timesteps=False)
    walking_model.save("basic_walking_model")
    print("✅ Walking model trained!")
    
    return walking_model

# TRAINING METHOD 2: QUICK IMPROVEMENT
# Use this if you already have a model and want to improve it quickly

def quick_improvement():
    """Quick improvement of existing walking model"""
    print("🔥 QUICK IMPROVEMENT TRAINING")
    print("=" * 40)
    
    # Try to load existing model
    env = DummyVecEnv([lambda: Monitor(WalkingHumanoidEnv())])
    
    try:
        model = PPO.load("basic_walking_model", env=env)
        print("✅ Loaded basic walking model")
    except:
        try:
            model = PPO.load("walking_model", env=env)
            print("✅ Loaded previous walking model")
        except:
            print("❌ No existing model found. Run basic_training() first!")
            return None
    
    # Fine-tune with lower learning rate
    model.learning_rate = 1e-4
    print(f"🎯 Fine-tuning with learning rate: {model.learning_rate}")
    
    model.learn(total_timesteps=150000, progress_bar=True, 
               tb_log_name="quick_improve", reset_num_timesteps=False)
    model.save("quick_improved_model")
    print("✅ Quick improvement completed!")
    
    return model

# TRAINING METHOD 3: CURRICULUM LEARNING (BEST RESULTS)
# Advanced method that starts easy and gets progressively harder

def curriculum_training():
    """Advanced curriculum learning - start easy, get harder"""
    print("🎓 CURRICULUM LEARNING TRAINING")
    print("=" * 50)
    
    difficulties = [0.5, 0.7, 1.0, 1.3, 1.5]  # Easy to hard
    model = None
    
    for i, difficulty in enumerate(difficulties):
        print(f"\n🎯 STAGE {i+1}/5: Difficulty {difficulty}")
        
        # Create environment with current difficulty
        env = DummyVecEnv([lambda d=difficulty: Monitor(ImprovedWalkingEnv(difficulty=d))])
        
        if model is not None:
            model.set_env(env)
            print(f"✅ Continuing from previous stage")
        else:
            model = PPO(
                "MlpPolicy", env,
                learning_rate=3e-4 / difficulty,
                n_steps=2048, batch_size=64, n_epochs=10,
                gamma=0.99, gae_lambda=0.95, clip_range=0.2,
                ent_coef=0.01 / difficulty, verbose=1,
                tensorboard_log="./logs/curriculum/",
                policy_kwargs=dict(net_arch=dict(pi=[512, 512, 256], vf=[512, 512, 256]))
            )
            print(f"✅ Created model for stage {i+1}")
        
        # Train for this stage
        timesteps = int(50000 * (1.0 + difficulty))
        print(f"Training for {timesteps} timesteps...")
        
        model.learn(total_timesteps=timesteps, progress_bar=True,
                   tb_log_name=f"curriculum_stage_{i+1}_diff_{difficulty}",
                   reset_num_timesteps=False)
        
        model.save(f"models/curriculum_stage_{i+1}")
        
        # Quick test
        print(f"🧪 Testing stage {i+1}...")
        test_env = ImprovedWalkingEnv(difficulty=difficulty)
        obs, _ = test_env.reset()
        total_reward = 0
        
        for step in range(200):
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, terminated, truncated, _ = test_env.step(action)
            total_reward += reward
            if terminated or truncated:
                break
        
        avg_reward = total_reward / (step + 1)
        distance = test_env.data.qpos[0]
        print(f"📊 Stage {i+1}: Avg reward={avg_reward:.2f}, Distance={distance:.2f}m")
    
    model.save("curriculum_final_model")
    print("✅ Curriculum training completed!")
    return model

print("✅ All training methods defined!")
print("\n🎯 CHOOSE YOUR TRAINING APPROACH:")
print("1. basic_training()      - Simple, reliable (30 min)")
print("2. quick_improvement()   - Fast improvement (20 min)")
print("3. curriculum_training() - Best results (60 min)")
print("\nRun one of these functions to start training!")


Output()

🎯 Starting Phase 1 training...
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 145      |
|    ep_rew_mean     | 173      |
| time/              |          |
|    fps             | 6453     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 1024     |
---------------------------------


----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 154        |
|    ep_rew_mean          | 184        |
| time/                   |            |
|    fps                  | 3822       |
|    iterations           | 2          |
|    time_elapsed         | 0          |
|    total_timesteps      | 2048       |
| train/                  |            |
|    approx_kl            | 0.03220638 |
|    clip_fraction        | 0.338      |
|    clip_range           | 0.2        |
|    entropy_loss         | -29.8      |
|    explained_variance   | -0.0174    |
|    learning_rate        | 0.0003     |
|    loss                 | 220        |
|    n_updates            | 5          |
|    policy_gradient_loss | -0.063     |
|    std                  | 1          |
|    value_loss           | 615        |
----------------------------------------


----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 153        |
|    ep_rew_mean          | 210        |
| time/                   |            |
|    fps                  | 3535       |
|    iterations           | 3          |
|    time_elapsed         | 0          |
|    total_timesteps      | 3072       |
| train/                  |            |
|    approx_kl            | 0.02677293 |
|    clip_fraction        | 0.273      |
|    clip_range           | 0.2        |
|    entropy_loss         | -29.8      |
|    explained_variance   | 0.34       |
|    learning_rate        | 0.0003     |
|    loss                 | 287        |
|    n_updates            | 10         |
|    policy_gradient_loss | -0.0611    |
|    std                  | 0.999      |
|    value_loss           | 668        |
----------------------------------------


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 153         |
|    ep_rew_mean          | 207         |
| time/                   |             |
|    fps                  | 3370        |
|    iterations           | 4           |
|    time_elapsed         | 1           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.024350924 |
|    clip_fraction        | 0.249       |
|    clip_range           | 0.2         |
|    entropy_loss         | -29.8       |
|    explained_variance   | 0.507       |
|    learning_rate        | 0.0003      |
|    loss                 | 342         |
|    n_updates            | 15          |
|    policy_gradient_loss | -0.0548     |
|    std                  | 0.999       |
|    value_loss           | 703         |
-----------------------------------------


...

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 220         |
|    ep_rew_mean          | 428         |
| time/                   |             |
|    fps                  | 2939        |
|    iterations           | 46          |
|    time_elapsed         | 16          |
|    total_timesteps      | 47104       |
| train/                  |             |
|    approx_kl            | 0.048108853 |
|    clip_fraction        | 0.36        |
|    clip_range           | 0.2         |
|    entropy_loss         | -29.4       |
|    explained_variance   | 0.981       |
|    learning_rate        | 0.0003      |
|    loss                 | 16.1        |
|    n_updates            | 225         |
|    policy_gradient_loss | -0.0577     |
|    std                  | 0.983       |
|    value_loss           | 62.1        |
-----------------------------------------


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 221         |
|    ep_rew_mean          | 431         |
| time/                   |             |
|    fps                  | 2939        |
|    iterations           | 47          |
|    time_elapsed         | 16          |
|    total_timesteps      | 48128       |
| train/                  |             |
|    approx_kl            | 0.041228235 |
|    clip_fraction        | 0.329       |
|    clip_range           | 0.2         |
|    entropy_loss         | -29.4       |
|    explained_variance   | 0.982       |
|    learning_rate        | 0.0003      |
|    loss                 | 12.9        |
|    n_updates            | 230         |
|    policy_gradient_loss | -0.0581     |
|    std                  | 0.983       |
|    value_loss           | 62.2        |
-----------------------------------------


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 222         |
|    ep_rew_mean          | 434         |
| time/                   |             |
|    fps                  | 2943        |
|    iterations           | 48          |
|    time_elapsed         | 16          |
|    total_timesteps      | 49152       |
| train/                  |             |
|    approx_kl            | 0.030495314 |
|    clip_fraction        | 0.276       |
|    clip_range           | 0.2         |
|    entropy_loss         | -29.4       |
|    explained_variance   | 0.979       |
|    learning_rate        | 0.0003      |
|    loss                 | 21.9        |
|    n_updates            | 235         |
|    policy_gradient_loss | -0.0537     |
|    std                  | 0.982       |
|    value_loss           | 72.4        |
-----------------------------------------


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 222         |
|    ep_rew_mean          | 438         |
| time/                   |             |
|    fps                  | 2942        |
|    iterations           | 49          |
|    time_elapsed         | 17          |
|    total_timesteps      | 50176       |
| train/                  |             |
|    approx_kl            | 0.032849446 |
|    clip_fraction        | 0.277       |
|    clip_range           | 0.2         |
|    entropy_loss         | -29.4       |
|    explained_variance   | 0.965       |
|    learning_rate        | 0.0003      |
|    loss                 | 19.6        |
|    n_updates            | 240         |
|    policy_gradient_loss | -0.0547     |
|    std                  | 0.981       |
|    value_loss           | 82          |
-----------------------------------------


✅ Phase 1 completed - Standing model saved!

🧪 Testing standing model...
Step   0: Height=1.30, Reward=3.10
Step 100: Height=1.19, Reward=2.10
Step 200: Height=1.09, Reward=2.10
Standing test ended at step 278
📊 Standing Test Results:
  Average reward: 1.98
  Final height: 0.48
  Success: ✅ YES


In [ ]:
# ACTUALLY RUN TRAINING (Choose ONE of these)
# Uncomment the training method you want to use

# Option 1: Basic Training (recommended for first time)
# trained_model = basic_training()

# Option 2: Quick Improvement (if you have existing model)
# trained_model = quick_improvement()

# Option 3: Curriculum Learning (best results but takes longer)
# trained_model = curriculum_training()

print("🎯 TRAINING INSTRUCTIONS:")
print("1. Uncomment ONE of the training lines above")
print("2. Run this cell to start training")
print("3. Wait for training to complete")
print("4. Continue to next cell for testing")

print("\n💡 RECOMMENDATIONS:")
print("- First time: Use basic_training()")
print("- Have existing model: Use quick_improvement()")  
print("- Want best results: Use curriculum_training()")

print("\n📊 MONITOR TRAINING:")
print("Run in terminal: tensorboard --logdir=./logs/")
print("Then open: http://localhost:6006")

# Placeholder - uncomment one of the training methods above
trained_model = None
print("\n⚠️  No training method selected yet!")
print("Uncomment one of the training lines above to start.")


Output()


🚶 PHASE 2: Training Walking Model
⚠️  Creating new walking model
Using cpu device
🎯 Training walking...
📊 Larger actions (-0.5 to 0.5) for walking movement


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 170      |
|    ep_rew_mean     | 258      |
| time/              |          |
|    fps             | 6989     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 176         |
|    ep_rew_mean          | 271         |
| time/                   |             |
|    fps                  | 3811        |
|    iterations           | 2           |
|    time_elapsed         | 1           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.027547084 |
|    clip_fraction        | 0.345       |
|    clip_range           | 0.2         |
|    entropy_loss         | -29.8       |
|    explained_variance   | -0.00697    |
|    learning_rate        | 0.0002      |
|    loss                 | 180         |
|    n_updates            | 8           |
|    policy_gradient_loss | -0.0676     |
|    std                  | 0.999       |
|    value_loss           | 722         |
-----------------------------------------


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 175         |
|    ep_rew_mean          | 282         |
| time/                   |             |
|    fps                  | 3164        |
|    iterations           | 3           |
|    time_elapsed         | 1           |
|    total_timesteps      | 6144        |
| train/                  |             |
|    approx_kl            | 0.023137331 |
|    clip_fraction        | 0.268       |
|    clip_range           | 0.2         |
|    entropy_loss         | -29.8       |
|    explained_variance   | 0.457       |
|    learning_rate        | 0.0002      |
|    loss                 | 297         |
|    n_updates            | 16          |
|    policy_gradient_loss | -0.0572     |
|    std                  | 0.998       |
|    value_loss           | 785         |
-----------------------------------------


...

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 248         |
|    ep_rew_mean          | 757         |
| time/                   |             |
|    fps                  | 2505        |
|    iterations           | 46          |
|    time_elapsed         | 37          |
|    total_timesteps      | 94208       |
| train/                  |             |
|    approx_kl            | 0.034643047 |
|    clip_fraction        | 0.365       |
|    clip_range           | 0.2         |
|    entropy_loss         | -29         |
|    explained_variance   | 0.981       |
|    learning_rate        | 0.0002      |
|    loss                 | 17          |
|    n_updates            | 360         |
|    policy_gradient_loss | -0.0606     |
|    std                  | 0.963       |
|    value_loss           | 91.9        |
-----------------------------------------


----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 250        |
|    ep_rew_mean          | 760        |
| time/                   |            |
|    fps                  | 2507       |
|    iterations           | 47         |
|    time_elapsed         | 38         |
|    total_timesteps      | 96256      |
| train/                  |            |
|    approx_kl            | 0.03497222 |
|    clip_fraction        | 0.34       |
|    clip_range           | 0.2        |
|    entropy_loss         | -29        |
|    explained_variance   | 0.98       |
|    learning_rate        | 0.0002     |
|    loss                 | 29.2       |
|    n_updates            | 368        |
|    policy_gradient_loss | -0.0611    |
|    std                  | 0.963      |
|    value_loss           | 117        |
----------------------------------------


----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 248        |
|    ep_rew_mean          | 759        |
| time/                   |            |
|    fps                  | 2507       |
|    iterations           | 48         |
|    time_elapsed         | 39         |
|    total_timesteps      | 98304      |
| train/                  |            |
|    approx_kl            | 0.03960548 |
|    clip_fraction        | 0.366      |
|    clip_range           | 0.2        |
|    entropy_loss         | -29        |
|    explained_variance   | 0.977      |
|    learning_rate        | 0.0002     |
|    loss                 | 15         |
|    n_updates            | 376        |
|    policy_gradient_loss | -0.0632    |
|    std                  | 0.961      |
|    value_loss           | 86.3       |
----------------------------------------


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 249         |
|    ep_rew_mean          | 760         |
| time/                   |             |
|    fps                  | 2509        |
|    iterations           | 49          |
|    time_elapsed         | 39          |
|    total_timesteps      | 100352      |
| train/                  |             |
|    approx_kl            | 0.042626806 |
|    clip_fraction        | 0.386       |
|    clip_range           | 0.2         |
|    entropy_loss         | -29         |
|    explained_variance   | 0.979       |
|    learning_rate        | 0.0002      |
|    loss                 | 16.3        |
|    n_updates            | 384         |
|    policy_gradient_loss | -0.0644     |
|    std                  | 0.961       |
|    value_loss           | 70.5        |
-----------------------------------------


✅ Phase 2 completed - Walking model saved!


In [ ]:
# COMPREHENSIVE TESTING OF TRAINED MODELS
# Tests all available models and shows performance comparison

def test_model(model_name, env_class, difficulty=1.0, steps=500):
    """Test a specific model and return performance metrics"""
    try:
        env = env_class() if env_class != ImprovedWalkingEnv else env_class(difficulty=difficulty)
        model = PPO.load(model_name, env=env)
        
        obs, _ = env.reset()
        total_reward = 0
        episode_rewards = []
        distances = []
        episode_count = 0
        
        for i in range(steps):
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, terminated, truncated, _ = env.step(action)
            total_reward += reward
            
            if i % 100 == 0:
                pos = env.data.qpos
                vel = env.data.qvel
                print(f"  Step {i:3d}: Height={pos[2]:.2f}, X={pos[0]:.2f}, VelX={vel[0]:.2f}, R={reward:.2f}")
            
            if terminated or truncated:
                episode_count += 1
                episode_rewards.append(total_reward)
                distances.append(env.data.qpos[0])
                
                print(f"  🏁 Episode {episode_count}: Reward={total_reward:.1f}, Distance={env.data.qpos[0]:.2f}m")
                obs, _ = env.reset()
                total_reward = 0
                
                if episode_count >= 3:  # Test 3 episodes
                    break
        
        # Calculate metrics
        avg_reward = np.mean(episode_rewards) if episode_rewards else total_reward / steps
        avg_distance = np.mean(distances) if distances else env.data.qpos[0]
        max_distance = max(distances) if distances else env.data.qpos[0]
        
        return {
            'avg_reward': avg_reward,
            'avg_distance': avg_distance,
            'max_distance': max_distance,
            'episodes': episode_count
        }
        
    except Exception as e:
        print(f"  ❌ Error testing {model_name}: {str(e)[:50]}...")
        return None

def comprehensive_test():
    """Test all available models and compare performance"""
    print("🧪 COMPREHENSIVE MODEL TESTING")
    print("=" * 50)
    
    # List of models to test
    models_to_test = [
        ("basic_standing_model", StandingHumanoidEnv, "Basic Standing"),
        ("basic_walking_model", WalkingHumanoidEnv, "Basic Walking"),
        ("quick_improved_model", WalkingHumanoidEnv, "Quick Improved"),
        ("curriculum_final_model", ImprovedWalkingEnv, "Curriculum Final"),
        ("walking_model", WalkingHumanoidEnv, "Previous Walking"),  # If exists
    ]
    
    results = []
    
    for model_name, env_class, description in models_to_test:
        print(f"\n🔍 Testing {description} ({model_name})...")
        
        result = test_model(model_name, env_class)
        if result:
            result['name'] = description
            result['model_file'] = model_name
            results.append(result)
            
            print(f"📊 {description} Results:")
            print(f"   Average reward: {result['avg_reward']:.2f}")
            print(f"   Average distance: {result['avg_distance']:.2f}m")
            print(f"   Max distance: {result['max_distance']:.2f}m")
            print(f"   Episodes completed: {result['episodes']}")
    
    # Compare results
    if results:
        print("\n📈 PERFORMANCE COMPARISON")
        print("=" * 50)
        
        # Sort by average reward
        results.sort(key=lambda x: x['avg_reward'], reverse=True)
        
        print("Ranking by Average Reward:")
        for i, result in enumerate(results):
            print(f"{i+1}. {result['name']:20} | Reward: {result['avg_reward']:6.2f} | Distance: {result['max_distance']:5.2f}m")
        
        # Find best model
        best_model = results[0]
        print(f"\n🥇 BEST MODEL: {best_model['name']}")
        print(f"   File: {best_model['model_file']}")
        print(f"   Performance: {best_model['avg_reward']:.2f} avg reward, {best_model['max_distance']:.2f}m max distance")
        
        return best_model['model_file']
    else:
        print("❌ No models found to test. Train a model first!")
        return None

# Run comprehensive testing
best_model_name = comprehensive_test()

if best_model_name:
    print(f"\n✅ Testing completed! Best model: {best_model_name}")
    print("\n🎬 Ready for visualization in next cell!")
else:
    print("\n⚠️  No trained models found.")
    print("Go back and run one of the training methods first!")



🧪 Testing walking model...
Step   0: Height=1.30, X=-0.00, VelX=-0.00, Reward=3.17
Step 100: Height=1.22, X=0.02, VelX=0.29, Reward=3.76
Step 200: Height=1.12, X=-0.28, VelX=-0.99, Reward=3.00
Walking test ended at step 280

📊 Walking Test Results:
  Average reward: 2.63
  Final height: 0.40
  Distance traveled: -0.89 meters
  Success: ❌ NO
⚠️  May need more training or parameter tuning


In [ ]:
# 🎬 ULTIMATE HUMANOID VIEWER AND ANALYZER

def interactive_viewer(model_name, env_class=None, difficulty=1.0):
    """Interactive viewer with real-time stats and controls"""
    print(f"🎬 Loading {model_name} for interactive viewing...")
    
    try:
        # Auto-detect environment type
        if env_class is None:
            if "standing" in model_name.lower():
                env_class = StandingHumanoidEnv
            elif "curriculum" in model_name.lower():
                env_class = ImprovedWalkingEnv
            else:
                env_class = WalkingHumanoidEnv
        
        # Create environment
        env = env_class() if env_class != ImprovedWalkingEnv else env_class(difficulty=difficulty)
        model = PPO.load(model_name, env=env)
        
        print(f"✅ Loaded {model_name} successfully!")
        print("🎮 Interactive Controls:")
        print("   - Runs automatically")
        print("   - Stats printed every 100 steps")  
        print("   - Auto-restart on episode end")
        print("   - Press Ctrl+C in terminal to stop")
        
        obs, _ = env.reset()
        step_count = 0
        episode_count = 0
        total_reward = 0
        best_distance = 0
        
        try:
            while True:
                action, _ = model.predict(obs, deterministic=True)
                obs, reward, terminated, truncated, _ = env.step(action)
                total_reward += reward
                step_count += 1
                
                # Render
                try:
                    env.render()
                except:
                    if step_count == 1:
                        print("⚠️  Visual rendering not available, showing stats only")
                
                # Print stats
                if step_count % 100 == 0:
                    pos = env.data.qpos
                    vel = env.data.qvel
                    best_distance = max(best_distance, pos[0])
                    
                    print(f"Step {step_count:4d} | Height: {pos[2]:5.2f} | X: {pos[0]:6.2f} | VelX: {vel[0]:5.2f} | Reward: {reward:6.2f}")
                
                # Episode management
                if terminated or truncated:
                    episode_count += 1
                    final_pos = env.data.qpos
                    
                    print(f"\n🏁 Episode {episode_count} Summary:")
                    print(f"   Steps: {step_count}")
                    print(f"   Total reward: {total_reward:.1f}")
                    print(f"   Distance: {final_pos[0]:.2f}m")
                    print(f"   Best distance so far: {best_distance:.2f}m")
                    print(f"   Average reward: {total_reward/step_count:.3f}")
                    print()
                    
                    obs, _ = env.reset()
                    step_count = 0
                    total_reward = 0
                    time.sleep(1.0)  # Brief pause between episodes
                
                time.sleep(0.02)  # 50 FPS
                
        except KeyboardInterrupt:
            print("\n🛑 Viewer stopped by user")
            print(f"📊 Final Stats: {episode_count} episodes, best distance: {best_distance:.2f}m")
    
    except Exception as e:
        print(f"❌ Error loading {model_name}: {e}")
        print("Available models:")
        available_models = ["basic_standing_model", "basic_walking_model", "quick_improved_model", 
                          "curriculum_final_model", "walking_model"]
        for model in available_models:
            try:
                PPO.load(model)
                print(f"  ✅ {model}")
            except:
                print(f"  ❌ {model}")

def quick_comparison_view():
    """Quickly view and compare multiple models"""
    print("⚡ QUICK COMPARISON VIEWER")
    print("=" * 30)
    
    models_to_compare = [
        ("basic_walking_model", WalkingHumanoidEnv),
        ("quick_improved_model", WalkingHumanoidEnv), 
        ("curriculum_final_model", ImprovedWalkingEnv),
    ]
    
    for model_name, env_class in models_to_compare:
        try:
            print(f"\n🔍 Quick test: {model_name}")
            env = env_class() if env_class != ImprovedWalkingEnv else env_class(difficulty=1.0)
            model = PPO.load(model_name, env=env)
            
            obs, _ = env.reset()
            total_reward = 0
            
            for i in range(200):
                action, _ = model.predict(obs, deterministic=True)
                obs, reward, terminated, truncated, _ = env.step(action)
                total_reward += reward
                
                if terminated or truncated:
                    break
            
            distance = env.data.qpos[0]
            avg_reward = total_reward / (i + 1)
            print(f"   📊 {model_name}: {avg_reward:.2f} avg reward, {distance:.2f}m distance")
            
        except:
            print(f"   ❌ {model_name}: Not available")

print("🎬 ULTIMATE HUMANOID VIEWER SUITE")
print("=" * 50)

print("\n🎮 OPTION 1: Interactive Viewer (Recommended)")
print("Choose a model to view interactively:")

# Check available models
available_models = []
model_options = [
    ("basic_standing_model", "Basic Standing"),
    ("basic_walking_model", "Basic Walking"), 
    ("quick_improved_model", "Quick Improved"),
    ("curriculum_final_model", "Curriculum Final"),
    ("walking_model", "Previous Walking")
]

for model_file, description in model_options:
    try:
        PPO.load(model_file)
        available_models.append((model_file, description))
        print(f"  ✅ {description}: interactive_viewer('{model_file}')")
    except:
        print(f"  ❌ {description}: Not trained yet")

print("\n⚡ OPTION 2: Quick Comparison")
print("  quick_comparison_view()  # Compare all models quickly")

print("\n🖥️  OPTION 3: Terminal Viewer (External)")
print("Run this in terminal for any model:")
print("mjpython -c \"")
print("import sys; sys.path.append('.')") 
print("exec(open('train_humanoid.ipynb').read())  # Load notebook functions")
print("interactive_viewer('basic_walking_model')  # Replace with your model")
print("\"")

print("\n💡 USAGE EXAMPLES:")
if available_models:
    best_model = available_models[0][0]  # First available model
    print(f"interactive_viewer('{best_model}')  # View best available model")
    print(f"quick_comparison_view()  # Compare all models")
else:
    print("No models available yet. Train a model first!")

print("\n📊 EXPECTED IMPROVEMENTS:")
print("✅ Positive rewards (3-7+ instead of negative)")
print("✅ Forward walking (2-5+ meters per episode)")  
print("✅ Stable upright posture (height > 1.0m)")
print("✅ Consistent walking behavior")
print("🚫 No more 'flying' or dimensional issues!")

# Auto-run quick comparison if models exist
if available_models:
    print("\n🚀 Running quick comparison of available models...")
    quick_comparison_view()
else:
    print("\n⚠️  No trained models found!")
    print("Go back and uncomment a training method in the previous cell.")


🎬 How to view your trained walking humanoid:

💻 OPTION 1: Direct Python viewer
Copy and run this code in a new cell:

# Quick inline viewer
test_env = WalkingHumanoidEnv()
obs, _ = test_env.reset()

for i in range(1000):
    action, _ = walking_model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, _ = test_env.step(action)
    test_env.render()  # This will open MuJoCo viewer
    time.sleep(0.02)

    if terminated or truncated:
        obs, _ = test_env.reset()
        print(f"Episode restarted at step {i}")

    if i % 200 == 0:
        pos = test_env.data.qpos
        print(f"Step {i}: Height={pos[2]:.2f}, Distance={pos[0]:.2f}")


🖥️  OPTION 2: Terminal viewer (recommended)
Run this command in terminal:
mjpython -c "
from simple_stand_walk import WalkingHumanoidEnv; from stable_baselines3 import PPO; import time
env = WalkingHumanoidEnv(); model = PPO.load('walking_model', env=env)
obs, _ = env.reset()
for i in range(2000):
    action, _ = model.predict(obs

In [10]:
# CORRECT: Use the NEW working models
from simple_stand_walk import WalkingHumanoidEnv
from stable_baselines3 import PPO

# Create the NEW environment (no dimensional issues)
test_env = WalkingHumanoidEnv()

# Load the NEW walking model
walking_model = PPO.load("walking_model", env=test_env)

obs, _ = test_env.reset()

for i in range(1000):
    action, _ = walking_model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, _ = test_env.step(action)
    test_env.render()  # This will show PROPER standing/walking
    
    if i % 100 == 0:
        pos = test_env.data.qpos
        print(f"Step {i}: Height={pos[2]:.2f}, Distance={pos[0]:.2f}, Reward={reward:.2f}")
    
    if terminated or truncated:
        obs, _ = test_env.reset()
        print(f"Episode restarted at step {i}")
    
    time.sleep(0.02)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Step 0: Height=1.33, Distance=0.02, Reward=3.16
Step 100: Height=1.18, Distance=0.13, Reward=3.09
Step 200: Height=1.11, Distance=0.31, Reward=5.37
Episode restarted at step 273
Step 300: Height=1.27, Distance=0.04, Reward=4.37
Step 400: Height=1.18, Distance=-0.01, Reward=3.14
Step 500: Height=1.05, Distance=-0.36, Reward=1.99
Episode restarted at step 583
Step 600: Height=1.29, Distance=-0.01, Reward=3.74
Step 700: Height=1.19, Distance=0.10, Reward=3.16
Step 800: Height=1.00, Distance=0.42, Reward=4.94
Episode restarted at step 860
Step 900: Height=1.25, Distance=0.08, Reward=4.00


In [ ]:
# CORRECT: Use the NEW working models
from simple_stand_walk import WalkingHumanoidEnv
from stable_baselines3 import PPO

# Create the NEW environment (no dimensional issues)
test_env = WalkingHumanoidEnv()

# Load the NEW walking model
walking_model = PPO.load("walking_model", env=test_env)

obs, _ = test_env.reset()

for i in range(1000):
    action, _ = walking_model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, _ = test_env.step(action)
    test_env.render()  # This will show PROPER standing/walking
    
    if i % 100 == 0:
        pos = test_env.data.qpos
        print(f"Step {i}: Height={pos[2]:.2f}, Distance={pos[0]:.2f}, Reward={reward:.2f}")
    
    if terminated or truncated:
        obs, _ = test_env.reset()
        print(f"Episode restarted at step {i}")
    
    time.sleep(0.02)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Step 0: Height=1.33, Distance=0.02, Reward=3.16
Step 100: Height=1.18, Distance=0.13, Reward=3.09
Step 200: Height=1.11, Distance=0.31, Reward=5.37
Episode restarted at step 273
Step 300: Height=1.27, Distance=0.04, Reward=4.37
Step 400: Height=1.18, Distance=-0.01, Reward=3.14
Step 500: Height=1.05, Distance=-0.36, Reward=1.99
Episode restarted at step 583
Step 600: Height=1.29, Distance=-0.01, Reward=3.74
Step 700: Height=1.19, Distance=0.10, Reward=3.16
Step 800: Height=1.00, Distance=0.42, Reward=4.94
Episode restarted at step 860
Step 900: Height=1.25, Distance=0.08, Reward=4.00
